In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from datetime import datetime as dt
import itertools


from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm


from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

from keras.utils import np_utils

%matplotlib inline

Using TensorFlow backend.


# Load the Dataset

In [2]:
df = pd.read_csv("../datasets/FIFA 1990-2018 - Cleaned.csv")
squads = pd.read_csv("../datasets/New_data/2018 FIFA World Cup Squads.csv")
fifa18 = pd.read_csv("../datasets/New_data/complete.csv")
distance = pd.read_csv("../datasets/New_data/distance.csv", error_bad_lines=False)
fifa_ranking = pd.read_csv("../datasets/New_data/fifa_ranking.csv")

b'Skipping line 31: expected 4 fields, saw 5\nSkipping line 63: expected 4 fields, saw 5\nSkipping line 85: expected 4 fields, saw 5\nSkipping line 102: expected 4 fields, saw 5\nSkipping line 173: expected 4 fields, saw 5\nSkipping line 174: expected 4 fields, saw 5\nSkipping line 196: expected 4 fields, saw 5\nSkipping line 203: expected 4 fields, saw 5\nSkipping line 204: expected 4 fields, saw 5\nSkipping line 207: expected 4 fields, saw 5\nSkipping line 211: expected 4 fields, saw 5\nSkipping line 213: expected 4 fields, saw 5\n'


# Select data for 2018

In [3]:
ranking_2018 = fifa_ranking[(fifa_ranking['rank_date']>"2018-04-01") & (fifa_ranking['rank_date']<"2018-05-01")]
df=df[df.Year == 2018]
ranking_2018.shape

(211, 17)

# change names

In [4]:
squads=squads.replace({"South Korea":"Korea Republic"})
squads=squads.replace({"Korea Republic":"Korea Republic"})
squads = squads.replace({"Iran":"IR Iran"})

# Duplicate

In [5]:
# we should change all respective column for duplicate
for index,row in df.iterrows():
    df.set_value(index+64,["Team1","Team2","Rounds","Stadium","City","Time","Day","Month","Year","Score1","Score2","Remarks"],[row["Team2"],row["Team1"],row["Rounds"],row['Stadium'],row['City'],row['Time'],row["Day"],row["Month"],row["Year"],row["Score2"],row["Score1"],row['Remarks']])
df.shape

/home/milad/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until


(128, 12)

In [6]:
# Create new columns with ranking position of each team
df.insert(2, 'rank_Team1', df['Team1'].map(ranking_2018.set_index('country_full')['rank']))
df.insert(3, 'rank_Team2', df['Team2'].map(ranking_2018.set_index('country_full')['rank']))

df.insert(4, 'rank_change_Team1', df['Team1'].map(ranking_2018.set_index('country_full')['rank_change']))
df.insert(5, 'rank_change_Team2', df['Team2'].map(ranking_2018.set_index('country_full')['rank_change']))

In [7]:
df.isnull().sum()

Team1                  0
Team2                  0
rank_Team1             0
rank_Team2             0
rank_change_Team1      0
rank_change_Team2      0
Rounds                 0
Stadium                0
City                   0
Time                   0
Day                    0
Month                  0
Year                   0
Score1                 0
Score2                 0
Remarks              100
dtype: int64

# Add Team features

In [8]:
# #AVG Age of each team in each position
age_avg=squads.groupby(["Team","Position"])['Age'].sum()/squads.groupby(["Team","Position"])['Age'].size()
age_avg = age_avg.to_frame()

age_avg['Team_name'] = age_avg.index.get_level_values(0).tolist()
age_avg['pos'] = age_avg.index.get_level_values(1).tolist()

In [9]:
age_DF = age_avg.loc[age_avg['pos'] == 'DF']
age_FW = age_avg.loc[age_avg['pos'] == 'FW']
age_GK = age_avg.loc[age_avg['pos'] == 'GK']
age_MF = age_avg.loc[age_avg['pos'] == 'MF']

In [ ]:
df.head()

In [10]:
df.insert(2, 'Team1_avg_age_DF', df['Team1'].map(age_DF.set_index('Team_name')['Age']))
df.insert(3, 'Team1_avg_age_FW', df['Team1'].map(age_FW.set_index('Team_name')['Age']))
df.insert(4, 'Team1_avg_age_GK', df['Team1'].map(age_GK.set_index('Team_name')['Age']))
df.insert(5, 'Team1_avg_age_MF', df['Team1'].map(age_MF.set_index('Team_name')['Age']))

In [11]:
df.insert(6, 'Team2_avg_age_DF', df['Team2'].map(age_DF.set_index('Team_name')['Age']))
df.insert(7, 'Team2_avg_age_FW', df['Team2'].map(age_FW.set_index('Team_name')['Age']))
df.insert(8, 'Team2_avg_age_GK', df['Team2'].map(age_GK.set_index('Team_name')['Age']))
df.insert(9, 'Team2_avg_age_MF', df['Team2'].map(age_MF.set_index('Team_name')['Age']))

In [12]:
df.isnull().sum()

Team1                  0
Team2                  0
Team1_avg_age_DF       0
Team1_avg_age_FW       0
Team1_avg_age_GK       0
Team1_avg_age_MF       0
Team2_avg_age_DF       0
Team2_avg_age_FW       0
Team2_avg_age_GK       0
Team2_avg_age_MF       0
rank_Team1             0
rank_Team2             0
rank_change_Team1      0
rank_change_Team2      0
Rounds                 0
Stadium                0
City                   0
Time                   0
Day                    0
Month                  0
Year                   0
Score1                 0
Score2                 0
Remarks              100
dtype: int64

In [13]:
#AVG Caps of each team per position
caps_avg = squads.groupby(["Team","Position"])['Caps'].sum()/squads.groupby(["Team","Position"])['Caps'].size()
caps_avg = caps_avg.to_frame()

caps_avg['Team_name'] = caps_avg.index.get_level_values(0).tolist()
caps_avg['pos'] = caps_avg.index.get_level_values(1).tolist()

In [14]:
# caps_avg = caps_avg.replace({"Iran":"IR Iran"})
# caps_avg = caps_avg.replace({"Korea Republic":"South Korea"})
# caps_avg.head()

In [15]:
caps_DF = caps_avg.loc[age_avg['pos'] == 'DF']
caps_FW = caps_avg.loc[age_avg['pos'] == 'FW']
caps_GK = caps_avg.loc[age_avg['pos'] == 'GK']
caps_MF = caps_avg.loc[age_avg['pos'] == 'MF']

In [16]:
df.insert(10, 'Team1_avg_caps_DF', df['Team1'].map(caps_DF.set_index('Team_name')['Caps']))
df.insert(11, 'Team1_avg_caps_FW', df['Team1'].map(caps_FW.set_index('Team_name')['Caps']))
df.insert(12, 'Team1_avg_caps_GK', df['Team1'].map(caps_GK.set_index('Team_name')['Caps']))
df.insert(13, 'Team1_avg_caps_MF', df['Team1'].map(caps_MF.set_index('Team_name')['Caps']))

df.insert(14, 'Team2_avg_caps_DF', df['Team2'].map(caps_DF.set_index('Team_name')['Caps']))
df.insert(15, 'Team2_avg_caps_FW', df['Team2'].map(caps_FW.set_index('Team_name')['Caps']))
df.insert(16, 'Team2_avg_caps_GK', df['Team2'].map(caps_GK.set_index('Team_name')['Caps']))
df.insert(17, 'Team2_avg_caps_MF', df['Team2'].map(caps_MF.set_index('Team_name')['Caps']))

In [17]:
df.head()

,Team1,Team2,Team1_avg_age_DF,Team1_avg_age_FW,Team1_avg_age_GK,Team1_avg_age_MF,Team2_avg_age_DF,Team2_avg_age_FW,Team2_avg_age_GK,Team2_avg_age_MF,...,Rounds,Stadium,City,Time,Day,Month,Year,Score1,Score2,Remarks
0,Russia,Saudi Arabia,29.857143,28.500000,30.666667,26.545455,28.714286,28.333333,30.333333,27.000000,...,Group Phase,Luzhniki Stadium,Moscow,18:00,14.0,6.0,2018.0,5.0,0.0,NaN
1,Egypt,Uruguay,28.750000,25.833333,36.000000,26.833333,27.857143,27.666667,31.666667,25.714286,...,Group Phase,Ekaterinburg Arena,Ekaterinburg,17:00,15.0,6.0,2018.0,0.0,1.0,NaN
2,Morocco,IR Iran,27.000000,26.750000,26.000000,26.900000,26.500000,26.428571,26.333333,27.400000,...,Group Phase,Saint Petersburg Stadium,St. Petersburg,18:00,15.0,6.0,2018.0,0.0,1.0,NaN
3,Portugal,Spain,28.375000,26.600000,31.000000,27.000000,28.500000,28.800000,28.333333,26.714286,...,Group Phase,Fisht Stadium,Sochi,21:00,15.0,6.0,2018.0,3.0,3.0,{WinTeamName} win on aggregate after regular t...
4,France,Australia,24.375000,24.142857,29.666667,27.000000,27.428571,26.625000,31.666667,27.000000,...,Group Phase,Kazan Arena,Kazan,13:00,16.0,6.0,2018.0,2.0,1.0,NaN


In [18]:
df.isnull().sum()

Team1                  0
Team2                  0
Team1_avg_age_DF       0
Team1_avg_age_FW       0
Team1_avg_age_GK       0
Team1_avg_age_MF       0
Team2_avg_age_DF       0
Team2_avg_age_FW       0
Team2_avg_age_GK       0
Team2_avg_age_MF       0
Team1_avg_caps_DF      0
Team1_avg_caps_FW      0
Team1_avg_caps_GK      0
Team1_avg_caps_MF      0
Team2_avg_caps_DF      0
Team2_avg_caps_FW      0
Team2_avg_caps_GK      0
Team2_avg_caps_MF      0
rank_Team1             0
rank_Team2             0
rank_change_Team1      0
rank_change_Team2      0
Rounds                 0
Stadium                0
City                   0
Time                   0
Day                    0
Month                  0
Year                   0
Score1                 0
Score2                 0
Remarks              100
dtype: int64

# change Rounds

In [19]:
rounds = df.Rounds
rounds.unique()
rounds_dict = {"Group Phase":1, "Round of 16":2, "Quarter Finals":3, "Semi Finals":4, "Play-off for third place":5, "Finals":6}
rounds_encoded = []

for i in range(len(df)):
    rounds_encoded.append(rounds_dict[list(df.Rounds)[i]])
    
df["Rounds"] = rounds_encoded

In [20]:
df.head()

,Team1,Team2,Team1_avg_age_DF,Team1_avg_age_FW,Team1_avg_age_GK,Team1_avg_age_MF,Team2_avg_age_DF,Team2_avg_age_FW,Team2_avg_age_GK,Team2_avg_age_MF,...,Rounds,Stadium,City,Time,Day,Month,Year,Score1,Score2,Remarks
0,Russia,Saudi Arabia,29.857143,28.500000,30.666667,26.545455,28.714286,28.333333,30.333333,27.000000,...,1,Luzhniki Stadium,Moscow,18:00,14.0,6.0,2018.0,5.0,0.0,NaN
1,Egypt,Uruguay,28.750000,25.833333,36.000000,26.833333,27.857143,27.666667,31.666667,25.714286,...,1,Ekaterinburg Arena,Ekaterinburg,17:00,15.0,6.0,2018.0,0.0,1.0,NaN
2,Morocco,IR Iran,27.000000,26.750000,26.000000,26.900000,26.500000,26.428571,26.333333,27.400000,...,1,Saint Petersburg Stadium,St. Petersburg,18:00,15.0,6.0,2018.0,0.0,1.0,NaN
3,Portugal,Spain,28.375000,26.600000,31.000000,27.000000,28.500000,28.800000,28.333333,26.714286,...,1,Fisht Stadium,Sochi,21:00,15.0,6.0,2018.0,3.0,3.0,{WinTeamName} win on aggregate after regular t...
4,France,Australia,24.375000,24.142857,29.666667,27.000000,27.428571,26.625000,31.666667,27.000000,...,1,Kazan Arena,Kazan,13:00,16.0,6.0,2018.0,2.0,1.0,NaN


# Player Features

In [21]:
# fifa18 = fifa18.replace({"South Korea":"Korea Republic"})
wc_player_stats_18 = fifa18[fifa18["name"].isin(squads.Player) | fifa18["full_name"].isin(squads.Player)]
wc_fifa18_stats = fifa18[fifa18.nationality.isin(squads.Team)]

In [22]:
wc_fifa18_stats.replace([False,"False",True,"True"],[0,0,1,1])
unused_var=[]
for i in fifa18.columns:
    if "prefers" in i:
        unused_var.append(i)
wc_fifa18_stats = wc_fifa18_stats.drop(["club_logo","ID","real_face","birth_date","flag","photo"],axis=1)
wc_fifa18_stats = wc_fifa18_stats.drop(unused_var,axis=1)
grouped_wc_fifa18_stats = wc_fifa18_stats.groupby(['nationality']).apply(lambda x: (x.sort_values('overall',ascending=False)).head(30).mean()).sort_values('potential',ascending=True)
grouped_wc_fifa18_stats.head()

,special,age,height_cm,weight_kg,eur_value,eur_wage,eur_release_clause,overall,potential,pac,...,playmaker_speciality,engine_speciality,distance_shooter_speciality,crosser_speciality,free_kick_specialist_speciality,tackling_speciality,tactician_speciality,acrobat_speciality,strength_speciality,clinical_finisher_speciality
nationality,,,,,,,,,,,,,,,,,,,,,
Panama,1617.375000,25.812500,181.750000,76.500000,8.378125e+05,3312.500000,1.344812e+06,65.812500,69.437500,66.062500,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.062500,0.0
Tunisia,1677.966667,27.266667,182.700000,76.666667,2.261167e+06,11700.000000,2.739000e+06,68.966667,71.533333,66.200000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.033333,0.0
Costa Rica,1695.966667,26.433333,181.000000,75.400000,3.196167e+06,18033.333333,5.612536e+06,69.900000,72.000000,73.400000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.066667,0.0
Iceland,1771.000000,27.233333,184.066667,78.200000,2.607500e+06,16700.000000,4.797357e+06,70.033333,72.166667,67.766667,...,0.0,0.1,0.0,0.033333,0.033333,0.0,0.0,0.000000,0.000000,0.0
Peru,1651.965517,25.517241,178.931034,73.896552,2.122069e+06,6620.689655,5.940867e+06,69.137931,72.689655,69.551724,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.034483,0.000000,0.0


In [23]:
grouped_wc_fifa18_stats['Team_name'] = grouped_wc_fifa18_stats.index.get_level_values(0).tolist()

In [24]:
grouped_wc_fifa18_stats = grouped_wc_fifa18_stats.replace({"Iran":"IR Iran"})
grouped_wc_fifa18_stats = grouped_wc_fifa18_stats.replace({"South Korea":"Korea Republic"})
# grouped_wc_fifa18_stats.head()

In [25]:
df.isnull().sum()

Team1                  0
Team2                  0
Team1_avg_age_DF       0
Team1_avg_age_FW       0
Team1_avg_age_GK       0
Team1_avg_age_MF       0
Team2_avg_age_DF       0
Team2_avg_age_FW       0
Team2_avg_age_GK       0
Team2_avg_age_MF       0
Team1_avg_caps_DF      0
Team1_avg_caps_FW      0
Team1_avg_caps_GK      0
Team1_avg_caps_MF      0
Team2_avg_caps_DF      0
Team2_avg_caps_FW      0
Team2_avg_caps_GK      0
Team2_avg_caps_MF      0
rank_Team1             0
rank_Team2             0
rank_change_Team1      0
rank_change_Team2      0
Rounds                 0
Stadium                0
City                   0
Time                   0
Day                    0
Month                  0
Year                   0
Score1                 0
Score2                 0
Remarks              100
dtype: int64

In [26]:
for key in grouped_wc_fifa18_stats.keys():
    if key != 'Team_name':
        df.insert(12, 'Team2_'+key, df['Team2'].map(grouped_wc_fifa18_stats.set_index('Team_name')[key]))
        df.insert(12, 'Team1_'+key, df['Team1'].map(grouped_wc_fifa18_stats.set_index('Team_name')[key]))
        

In [27]:
df.head()

,Team1,Team2,Team1_avg_age_DF,Team1_avg_age_FW,Team1_avg_age_GK,Team1_avg_age_MF,Team2_avg_age_DF,Team2_avg_age_FW,Team2_avg_age_GK,Team2_avg_age_MF,...,Rounds,Stadium,City,Time,Day,Month,Year,Score1,Score2,Remarks
0,Russia,Saudi Arabia,29.857143,28.500000,30.666667,26.545455,28.714286,28.333333,30.333333,27.000000,...,1,Luzhniki Stadium,Moscow,18:00,14.0,6.0,2018.0,5.0,0.0,NaN
1,Egypt,Uruguay,28.750000,25.833333,36.000000,26.833333,27.857143,27.666667,31.666667,25.714286,...,1,Ekaterinburg Arena,Ekaterinburg,17:00,15.0,6.0,2018.0,0.0,1.0,NaN
2,Morocco,IR Iran,27.000000,26.750000,26.000000,26.900000,26.500000,26.428571,26.333333,27.400000,...,1,Saint Petersburg Stadium,St. Petersburg,18:00,15.0,6.0,2018.0,0.0,1.0,NaN
3,Portugal,Spain,28.375000,26.600000,31.000000,27.000000,28.500000,28.800000,28.333333,26.714286,...,1,Fisht Stadium,Sochi,21:00,15.0,6.0,2018.0,3.0,3.0,{WinTeamName} win on aggregate after regular t...
4,France,Australia,24.375000,24.142857,29.666667,27.000000,27.428571,26.625000,31.666667,27.000000,...,1,Kazan Arena,Kazan,13:00,16.0,6.0,2018.0,2.0,1.0,NaN


In [28]:
df.isnull().sum()

Team1                                      0
Team2                                      0
Team1_avg_age_DF                           0
Team1_avg_age_FW                           0
Team1_avg_age_GK                           0
Team1_avg_age_MF                           0
Team2_avg_age_DF                           0
Team2_avg_age_FW                           0
Team2_avg_age_GK                           0
Team2_avg_age_MF                           0
Team1_avg_caps_DF                          0
Team1_avg_caps_FW                          0
Team1_clinical_finisher_speciality         3
Team2_clinical_finisher_speciality         3
Team1_strength_speciality                  3
Team2_strength_speciality                  3
Team1_acrobat_speciality                   3
Team2_acrobat_speciality                   3
Team1_tactician_speciality                 3
Team2_tactician_speciality                 3
Team1_tackling_speciality                  3
Team2_tackling_speciality                  3
Team1_free

# code country name

In [29]:
countries = list(set(df["Team1"]).union(set(df["Team2"])))
countries_dict = {}
for i in range(len(countries)):
    countries_dict[countries[i]] = i
    
Team1_encoded = []
Team2_encoded = []

for i in range(len(df)):
    Team1_encoded.append(countries_dict[list(df.Team1)[i]])
    Team2_encoded.append(countries_dict[list(df.Team2)[i]])
    
df["Team1"] = Team1_encoded
df["Team2"] = Team2_encoded

In [30]:
#Adding who is the winner 
winner = []
for i in range (len(df['Team1'])):
    if df['Score1'][i] > df['Score2'][i]:
        winner.append(df['Team1'][i])
    elif df['Score1'][i] < df['Score2'][i]:
        winner.append(df['Team2'][i])
    else:
        winner.append(-1)
df['winning_team'] = winner

df.head()

,Team1,Team2,Team1_avg_age_DF,Team1_avg_age_FW,Team1_avg_age_GK,Team1_avg_age_MF,Team2_avg_age_DF,Team2_avg_age_FW,Team2_avg_age_GK,Team2_avg_age_MF,...,Stadium,City,Time,Day,Month,Year,Score1,Score2,Remarks,winning_team
0,13,23,29.857143,28.500000,30.666667,26.545455,28.714286,28.333333,30.333333,27.000000,...,Luzhniki Stadium,Moscow,18:00,14.0,6.0,2018.0,5.0,0.0,NaN,13
1,20,10,28.750000,25.833333,36.000000,26.833333,27.857143,27.666667,31.666667,25.714286,...,Ekaterinburg Arena,Ekaterinburg,17:00,15.0,6.0,2018.0,0.0,1.0,NaN,10
2,12,11,27.000000,26.750000,26.000000,26.900000,26.500000,26.428571,26.333333,27.400000,...,Saint Petersburg Stadium,St. Petersburg,18:00,15.0,6.0,2018.0,0.0,1.0,NaN,11
3,5,3,28.375000,26.600000,31.000000,27.000000,28.500000,28.800000,28.333333,26.714286,...,Fisht Stadium,Sochi,21:00,15.0,6.0,2018.0,3.0,3.0,{WinTeamName} win on aggregate after regular t...,-1
4,6,17,24.375000,24.142857,29.666667,27.000000,27.428571,26.625000,31.666667,27.000000,...,Kazan Arena,Kazan,13:00,16.0,6.0,2018.0,2.0,1.0,NaN,6


In [31]:

df = df.drop(['Score1','Score2','Remarks',"Stadium","City"],axis=1)
df.head()

,Team1,Team2,Team1_avg_age_DF,Team1_avg_age_FW,Team1_avg_age_GK,Team1_avg_age_MF,Team2_avg_age_DF,Team2_avg_age_FW,Team2_avg_age_GK,Team2_avg_age_MF,...,rank_Team1,rank_Team2,rank_change_Team1,rank_change_Team2,Rounds,Time,Day,Month,Year,winning_team
0,13,23,29.857143,28.500000,30.666667,26.545455,28.714286,28.333333,30.333333,27.000000,...,66,70,-3,-1,1,18:00,14.0,6.0,2018.0,13
1,20,10,28.750000,25.833333,36.000000,26.833333,27.857143,27.666667,31.666667,25.714286,...,46,17,-2,5,1,17:00,15.0,6.0,2018.0,10
2,12,11,27.000000,26.750000,26.000000,26.900000,26.500000,26.428571,26.333333,27.400000,...,42,36,0,-3,1,18:00,15.0,6.0,2018.0,11
3,5,3,28.375000,26.600000,31.000000,27.000000,28.500000,28.800000,28.333333,26.714286,...,4,8,-1,-2,1,21:00,15.0,6.0,2018.0,-1
4,6,17,24.375000,24.142857,29.666667,27.000000,27.428571,26.625000,31.666667,27.000000,...,7,40,2,-3,1,13:00,16.0,6.0,2018.0,6


In [32]:
df.isnull().sum()

Team1                                    0
Team2                                    0
Team1_avg_age_DF                         0
Team1_avg_age_FW                         0
Team1_avg_age_GK                         0
Team1_avg_age_MF                         0
Team2_avg_age_DF                         0
Team2_avg_age_FW                         0
Team2_avg_age_GK                         0
Team2_avg_age_MF                         0
Team1_avg_caps_DF                        0
Team1_avg_caps_FW                        0
Team1_clinical_finisher_speciality       3
Team2_clinical_finisher_speciality       3
Team1_strength_speciality                3
Team2_strength_speciality                3
Team1_acrobat_speciality                 3
Team2_acrobat_speciality                 3
Team1_tactician_speciality               3
Team2_tactician_speciality               3
Team1_tackling_speciality                3
Team2_tackling_speciality                3
Team1_free_kick_specialist_speciality    3
Team2_free_

In [33]:
df = df.fillna(0)

In [ ]:
for c in df.keys():
    print(c)


In [34]:
df.shape

(128, 314)

In [35]:
df.head()

,Team1,Team2,Team1_avg_age_DF,Team1_avg_age_FW,Team1_avg_age_GK,Team1_avg_age_MF,Team2_avg_age_DF,Team2_avg_age_FW,Team2_avg_age_GK,Team2_avg_age_MF,...,rank_Team1,rank_Team2,rank_change_Team1,rank_change_Team2,Rounds,Time,Day,Month,Year,winning_team
0,13,23,29.857143,28.500000,30.666667,26.545455,28.714286,28.333333,30.333333,27.000000,...,66,70,-3,-1,1,18:00,14.0,6.0,2018.0,13
1,20,10,28.750000,25.833333,36.000000,26.833333,27.857143,27.666667,31.666667,25.714286,...,46,17,-2,5,1,17:00,15.0,6.0,2018.0,10
2,12,11,27.000000,26.750000,26.000000,26.900000,26.500000,26.428571,26.333333,27.400000,...,42,36,0,-3,1,18:00,15.0,6.0,2018.0,11
3,5,3,28.375000,26.600000,31.000000,27.000000,28.500000,28.800000,28.333333,26.714286,...,4,8,-1,-2,1,21:00,15.0,6.0,2018.0,-1
4,6,17,24.375000,24.142857,29.666667,27.000000,27.428571,26.625000,31.666667,27.000000,...,7,40,2,-3,1,13:00,16.0,6.0,2018.0,6


# Duplicate

In [36]:
df.loc[0,:]

Team1                                             13
Team2                                             23
Team1_avg_age_DF                             29.8571
Team1_avg_age_FW                                28.5
Team1_avg_age_GK                             30.6667
Team1_avg_age_MF                             26.5455
Team2_avg_age_DF                             28.7143
Team2_avg_age_FW                             28.3333
Team2_avg_age_GK                             30.3333
Team2_avg_age_MF                                  27
Team1_avg_caps_DF                            27.5714
Team1_avg_caps_FW                                 27
Team1_clinical_finisher_speciality                 0
Team2_clinical_finisher_speciality                 0
Team1_strength_speciality                  0.0666667
Team2_strength_speciality                  0.0333333
Team1_acrobat_speciality                           0
Team2_acrobat_speciality                   0.0333333
Team1_tactician_speciality                    

In [37]:
df.loc[64,:]

Team1                                             23
Team2                                             13
Team1_avg_age_DF                             28.7143
Team1_avg_age_FW                             28.3333
Team1_avg_age_GK                             30.3333
Team1_avg_age_MF                                  27
Team2_avg_age_DF                             29.8571
Team2_avg_age_FW                                28.5
Team2_avg_age_GK                             30.6667
Team2_avg_age_MF                             26.5455
Team1_avg_caps_DF                                 40
Team1_avg_caps_FW                            33.6667
Team1_clinical_finisher_speciality                 0
Team2_clinical_finisher_speciality                 0
Team1_strength_speciality                  0.0333333
Team2_strength_speciality                  0.0666667
Team1_acrobat_speciality                   0.0333333
Team2_acrobat_speciality                           0
Team1_tactician_speciality                    

# train & test

In [38]:
X_train = df.loc[df['Rounds'] == 1]
X_test = df.loc[df['Rounds'] != 1]



In [39]:
y_train = X_train["winning_team"]
y_test = X_test["winning_team"]

In [40]:
X_train = X_train.drop(["Rounds","winning_team","Time"],axis=1)
X_test = X_test.drop(["Rounds","winning_team","Time"],axis=1)

# train model

In [41]:
model = Sequential()
model.add(Dense(12, input_dim=X_train.shape[1], activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))
#model.dropout()
model.add(Dense(1, activation='sigmoid'))

In [42]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) #categorical crossentropy?

In [43]:
model.fit(X_train, y_train, validation_data=(X_test,y_test), shuffle=True, epochs=100, batch_size=64)

Train on 96 samples, validate on 32 samples
Epoch 1/100
96/96 [==============================] - 0s 3ms/step - loss: 6.5601 - acc: 0.0208 - val_loss: 5.3889 - val_acc: 0.0000e+00
Epoch 2/100
96/96 [==============================] - 0s 104us/step - loss: 6.3116 - acc: 0.0208 - val_loss: 5.1606 - val_acc: 0.0000e+00
Epoch 3/100
96/96 [==============================] - 0s 99us/step - loss: 6.0526 - acc: 0.0208 - val_loss: 4.9320 - val_acc: 0.0000e+00
Epoch 4/100
96/96 [==============================] - 0s 91us/step - loss: 5.8186 - acc: 0.0208 - val_loss: 4.7021 - val_acc: 0.0000e+00
Epoch 5/100
96/96 [==============================] - 0s 133us/step - loss: 5.5696 - acc: 0.0208 - val_loss: 4.4725 - val_acc: 0.0000e+00
Epoch 6/100
96/96 [==============================] - 0s 100us/step - loss: 5.3099 - acc: 0.0208 - val_loss: 4.2432 - val_acc: 0.0000e+00
Epoch 7/100
96/96 [==============================] - 0s 97us/step - loss: 5.0821 - acc: 0.0208 - val_loss: 4.0128 - val_acc: 0.0000e+00
Ep

Epoch 61/100
96/96 [==============================] - 0s 131us/step - loss: -8.4461 - acc: 0.0625 - val_loss: -8.4644 - val_acc: 0.0625
Epoch 62/100
96/96 [==============================] - 0s 118us/step - loss: -8.7057 - acc: 0.0625 - val_loss: -8.7045 - val_acc: 0.0625
Epoch 63/100
96/96 [==============================] - 0s 154us/step - loss: -8.9725 - acc: 0.0625 - val_loss: -8.9430 - val_acc: 0.0625
Epoch 64/100
96/96 [==============================] - 0s 115us/step - loss: -9.2243 - acc: 0.0625 - val_loss: -9.1811 - val_acc: 0.0625
Epoch 65/100
96/96 [==============================] - 0s 103us/step - loss: -9.4878 - acc: 0.0625 - val_loss: -9.4167 - val_acc: 0.0625
Epoch 66/100
96/96 [==============================] - 0s 111us/step - loss: -9.7423 - acc: 0.0625 - val_loss: -9.6510 - val_acc: 0.0625
Epoch 67/100
96/96 [==============================] - 0s 214us/step - loss: -10.0053 - acc: 0.0625 - val_loss: -9.8834 - val_acc: 0.0625
Epoch 68/100
96/96 [===========================

In [44]:
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

32/32 [==============================] - 0s 84us/step

acc: 6.25%


In [45]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

model= DecisionTreeClassifier()
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

accuracy_score(y_test, y_pred)

0.25

In [46]:
from sklearn import svm
model_svm = svm.SVC()
model.fit(X_train,y_train)

y_pred_svm = model.predict(X_test)

accuracy_score(y_test, y_pred_svm)

0.28125